# Institulac 

In [62]:
import json
import time
from datetime import datetime

import pandas as pd
pd.set_option("max_rows",100)
#pd.set_option("display.max_columns",100)
pd.set_option("max_colwidth",1000)

import helium as h
from selenium.common.exceptions import NoSuchElementException

In [63]:
# DICT CAT-PRODS-TAB
with open('dict_tables.json') as file_json:
    dict_tables=json.loads(file_json.read())

In [64]:
list_of_list_of_tables = []

In [65]:
# login =
# name_ins =
# usser =
# passw=

# login
browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
sleep=0.8

#browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
time.sleep(sleep)
h.click('Consulte Aquí')

time.sleep(sleep)
h.write('UNIVERSIDAD DE ANTIOQUIA',into='Digite el nombre de la Institución') # name ins

time.sleep(sleep)
h.click('Buscar')

time.sleep(sleep)
h.click(browser.find_element_by_id('list_instituciones'))

time.sleep(sleep)

time.sleep(sleep)
h.select('seleccione una','UNIVERSIDAD DE ANTIOQUIA') # name_ins

time.sleep(sleep)
h.write('annyarango',into='Usuario')                  # user

time.sleep(sleep)
h.write('1@Silver', into='Contraseña')                # passw

time.sleep(sleep)
h.click(h.Button('Ingresar'))

# cookie injection
time.sleep(sleep)
# implementation cookie injection

# get current cookie and store
new_cookie=browser.get_cookies()[0]
    
# create new_cookie with time_expire
time_expire = (datetime(2022,1,1) - datetime(1970,1,1)).total_seconds()
new_cookie['expiry'] = int(time_expire)
    
# delete cookie sites
browser.delete_all_cookies()
    
# add new cookie
browser.add_cookie(new_cookie)

# navigation 1
time.sleep(sleep)
h.click('Aval')

time.sleep(sleep)
h.click('Avalar grupos')

time.sleep(sleep)
h.click('Grupos Avalados')

# -- end login --

# list of total groups
#select max results per page
time.sleep(sleep)
h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'))

time.sleep(sleep)
h.select(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'),'100')

# catch 1: groups info [name, lider, cod,  link to producs]  
# schema
# empty df
# select max items per page
# while until end
# try:
    # catch table
    # preproces table
    # catch urls
    # add url colums
    # add df
    # click next page -> raise error
# except Nosuchelement:
    # break
    
# catch 1: list of groups
dfg=pd.DataFrame()
cont=True

while cont:
    
    try:
        # catch source
        time.sleep(sleep)
        source_g=browser.page_source
        
        # catch table
        time.sleep(sleep)
        df=pd.read_html(source_g, attrs={"id":"grupos_avalados"}, header=2)[0]
        
        # and preprocces it
        c=[x for x in df.columns if x.find('Unnamed:') == -1]
        dfgp=df[c][1:-1]
        print(dfgp.columns,dfgp.shape)
        
        # catch urls
        url=[a.get_attribute('href') for a in browser.find_elements_by_xpath('//table[@id="grupos_avalados"]//td[5]/a')]
        dfgp['Revisar'] = url
        dfg=dfg.append(dfgp)
        
        # click next page. this instruction rise error of the end. 
        h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//tr/td[3]/a'))
        
    except NoSuchElementException as e:
        
        print(e)
        print('out of cicle')
        break
        
    time.sleep(sleep)
    time.sleep(sleep)
    
dfg = dfg.reset_index(drop=True)
assert dfg.shape[0] == 324

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (24, 4)
Message: Unable to locate element: //table[@id="grupos_avalados"]//tr/td[3]/a

out of cicle


In [53]:
print(dfg['Nombre del grupo'][:1].values[0],'\n',dfg['Revisar'][:1].values[0])

print(dfg['Nombre del grupo'][-1:].values[0],'\n',dfg['Revisar'][-1:].values[0])

Grupo Tandem en Nano-bio-física 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000020456
Promoción de la Salud 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000001767


In [ ]:
time.sleep(sleep*2)

for u in dfg['Revisar']:
    # part info group
    
    # cathc II: INFO GROUP
    # ITEMS: info groups; members; products
    # for i in dfg[Revisar...] case i=0; before executing this cell locate at page 1 list groups

    # specific group url
    time.sleep(sleep)
    url_group = u

    # go to url group
    time.sleep(sleep)
    browser.get(url_group)

    # catch two tables: info grupo and  members
    source=browser.page_source

    #info
    l_info=pd.read_html(source, match='Nombre Grupo')
    info_g=l_info[3].pivot(columns=0,values=1)
    
    # store info group

    # members
    l_int = pd.read_html(source,attrs={'id':'tblIntegrantes'},header=2)
    mem_g=l_int[0]
    
    # store info members

    # Products

    time.sleep(sleep*5) # time time time !!!
    h.click(browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]'))

    # products by belongs to  # time time time
    time.sleep(sleep*7)       # time time time
    h.click(browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]'))

    time.sleep(sleep)
    url_products=browser.current_url


    # map all products, store those id categories that amount is different to 0 and id products asociated.
    # make queries with combinations of categories and products
    # make urls with diferent combinations of quieries
    # go to each of urls
    # load page source
    # catch table ( or tables) asociated with categories and products
    # store tables

    report = ''

    list_of_prods =[]

    # map all products and get products and subs diff to cero
    for i in browser.find_elements_by_xpath('//div[@id="accordionCatgP"]/h3'):

        report += i.text + '\n' 
        report += i.get_attribute('id') + '\n'     
        print(i.get_attribute('id'))

        time.sleep(sleep)
        # h.click(a)
        h.click(i)

        for j in browser.find_elements_by_xpath('//div[@aria-labelledby="%s"]/h3' % i.get_attribute('id')):

            report += '\t' + j.text + '\n' 
            report += '\t' + j.get_attribute('id') + '\n'

            if int(i.text[-2:-1]) > 0 and int(j.text[-2:-1]) > 0:

                list_of_prods.append([i.get_attribute('id'),j.get_attribute('id')])

        time.sleep(sleep) 
        # h.click(a)
        h.click(i)

    # print(report)
    # print('\n')
    # print('--------------------------------')
    time.sleep(sleep*2)
    
    tables=[]
    for p in range(len(list_of_prods)):

            # make query 
            query='categoriaP=%s&subcategoriaP=%s&aval=P' % (list_of_prods[p][0],list_of_prods[p][1])

            # make url query
            url_query = url_products.split('?')[0] + '?' + query + '&' + url_products.split('?')[1]

            # retrieve id asociated tables
            table_id = dict_tables[list_of_prods[p][0]][list_of_prods[p][1]]

            # go to url product by group
            time.sleep(sleep)
            print(url_query)
            browser.get(url_query)

            # load page
            time.sleep(sleep)
            page_source = browser.page_source

            # catch tables
            if isinstance(table_id,str): # case one table

                # catch title table
                title_table = browser.find_element_by_xpath('//div/p[@class="titulo_tabla"]').text 
                # cathc table
                print(table_id)
                time.sleep(sleep*2)
                table = pd.read_html(page_source,attrs={'id':table_id}, header=2)[0][1:-1]

                # store table
                tables.append(table)
                # ---- in building ----

            elif isinstance(table_id, list): # case multiple tables

                for i in range(len(table_id)):

                    # catch title specific table 
                    title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text

                    # catch table
                    table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[0][1:-1]

                    # store table
                    tables.append(table)
                    # -----------
    list_of_list_of_tables.append(tables)

NC_P
DTI_P
ASC_P
FRH_P
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=CAP_LIB_P&aval=P&idGrupo=00000000020456&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=Grupo%20Tandem%20en%20Nano-bio-f%EDsica&codIdGrupo=COL0202529
CAP_LIB_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=TES_MAST_P&aval=P&idGrupo=00000000020456&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=Grupo%20Tandem%20en%20Nano-bio-f%EDsica&codIdGrupo=COL0202529
TM_P_TABLE
NC_P
DTI_P
ASC_P
FRH_P
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=CON_INF_TEC_P&aval=P&idGrupo=00000000012069&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Psicolog%EDa%20Din%E1mica&codIdGrupo=COL0119469
CON_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=INF_TEC_P&aval=P&idGrupo=00000000012069&avalGr=T&codig

PID_P_TABLE
NC_P
DTI_P
ASC_P
FRH_P
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=ART_IMP_P&aval=P&idGrupo=00000000003658&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=MATHEMA-Formaci%F3n%20e%20Investigaci%F3n%20en%20Educaci%F3n%20Matem%E1tica&codIdGrupo=COL0053803
ART_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=ART_ELE_P&aval=P&idGrupo=00000000003658&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=MATHEMA-Formaci%F3n%20e%20Investigaci%F3n%20en%20Educaci%F3n%20Matem%E1tica&codIdGrupo=COL0053803
ART_E_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=LIB_P&aval=P&idGrupo=00000000003658&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=MATHEMA-Formaci%F3n%20e%20Investigaci%F3n%20en%20Educaci%F3n%20Matem%E1tica&codIdGrupo=COL0053803
LIB_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allp

CAP_LIB_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=GEN_CONT_IMP_P&aval=P&idGrupo=00000000008131&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Grupo%20de%20Investigaci%F3n%20en%20Historia%20Social&codIdGrupo=COL0009879
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=TES_PREG_P&aval=P&idGrupo=00000000008131&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Grupo%20de%20Investigaci%F3n%20en%20Historia%20Social&codIdGrupo=COL0009879
TP_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=PROY_INV_DES_P&aval=P&idGrupo=00000000008131&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Grupo%20de%20Investigaci%F3n%20en%20Historia%20Social&codIdGrupo=COL0009879
PID_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=PROY_INV_DES_INN_P&aval=P&idGrupo=000

## --------- ##

In [ ]:
# build excel here
xlswriter, pd.To
{}: "input -> proccess -> hojas"

    
# format header

In [61]:
time.sleep(sleep*2)

for u in dfg['Revisar']:
    
    # cathc II: INFO GROUP
    # ITEMS: info groups; members; products
    # for i in dfg[Revisar...] case i=0; before executing this cell locate at page 1 list groups

    # specific group url
    time.sleep(sleep)
    url_group = u

    # go to url group
    time.sleep(sleep)
    browser.get(url_group)

    # catch two tables: info grupo and  members
    source=browser.page_source

    #info
    l_info=pd.read_html(source, match='Nombre Grupo')
    info_g=l_info[3].pivot(columns=0,values=1)

    # members
    l_int = pd.read_html(source,attrs={'id':'tblIntegrantes'},header=2)
    mem_g=l_int[0]

    # productos

    time.sleep(sleep*5) # time time time !!!
    h.click(browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]'))

    # products by belongs to  # time time time
    time.sleep(sleep*7)
    h.click(browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]'))

    time.sleep(sleep)
    url_products=browser.current_url


    # map all products, store those id categories that amount is different to 0 and id products asociated.
    # make queries with combinations of categories and products
    # make urls with diferent combinations of quieries
    # go to each of urls
    # load page source
    # catch table ( or tables) asociated with categories and products
    # store tables

    report = ''

    list_of_prods =[]

    # map all products and get products and subs diff to cero
    for i in browser.find_elements_by_xpath('//div[@id="accordionCatgP"]/h3'):

        report += i.text + '\n' 
        report += i.get_attribute('id') + '\n'     
        print(i.get_attribute('id'))

        time.sleep(sleep)
        # h.click(a)
        h.click(i)

        for j in browser.find_elements_by_xpath('//div[@aria-labelledby="%s"]/h3' % i.get_attribute('id')):

            report += '\t' + j.text + '\n' 
            report += '\t' + j.get_attribute('id') + '\n'

            if int(i.text[-2:-1]) > 0 and int(j.text[-2:-1]) > 0:

                list_of_prods.append([i.get_attribute('id'),j.get_attribute('id')])

        time.sleep(sleep) 
        # h.click(a)
        h.click(i)

    # print(report)
    # print('\n')
    # print('--------------------------------')
    time.sleep(sleep*2)
    

NC_P
DTI_P
ASC_P
FRH_P
Nuevo Conocimiento (2)
NC_P
	Artículos publicados en revistas especializadas - Impresos (0)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (0)
	ART_ELE_P
	Libros resultado de investigación (0)
	LIB_P
	Capítulos en Libro resultado de investigación (2)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industri

NC_P
DTI_P
ASC_P
FRH_P
Nuevo Conocimiento (0)
NC_P
	Artículos publicados en revistas especializadas - Impresos (0)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (0)
	ART_ELE_P
	Libros resultado de investigación (0)
	LIB_P
	Capítulos en Libro resultado de investigación (0)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industri

NC_P
DTI_P
ASC_P
FRH_P
Nuevo Conocimiento (3)
NC_P
	Artículos publicados en revistas especializadas - Impresos (0)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (1)
	ART_ELE_P
	Libros resultado de investigación (1)
	LIB_P
	Capítulos en Libro resultado de investigación (1)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industri

NC_P
DTI_P
ASC_P
FRH_P
Nuevo Conocimiento (2)
NC_P
	Artículos publicados en revistas especializadas - Impresos (0)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (0)
	ART_ELE_P
	Libros resultado de investigación (0)
	LIB_P
	Capítulos en Libro resultado de investigación (2)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industri

KeyboardInterrupt: 

In [ ]:
print(report)

In [55]:
list_of_prods

[['NC_P', 'CAP_LIB_P'], ['FRH_P', 'TES_MAST_P']]

In [56]:
tables=[]
for p in range(len(list_of_prods)):

        # make query 
        query='categoriaP=%s&subcategoriaP=%s&aval=P' % (list_of_prods[p][0],list_of_prods[p][1])
        
        # make url query
        url_query = url_products.split('?')[0] + '?' + query + '&' + url_products.split('?')[1]

        # retrieve id asociated tables
        table_id = dict_tables[list_of_prods[p][0]][list_of_prods[p][1]]

        # go to url product by group
        time.sleep(sleep)
        print(url_query)
        browser.get(url_query)
        
        # load page
        time.sleep(sleep)
        page_source = browser.page_source

        # catch tables
        if isinstance(table_id,str): # case one table
            
            # catch title table
            title_table = browser.find_element_by_xpath('//div/p[@class="titulo_tabla"]').text 
            # cathc table
            print(table_id)
            time.sleep(sleep*2)
            table = pd.read_html(page_source,attrs={'id':table_id}, header=2)[0][1:-1]
            
            # store table
            tables.append(table)
            # ---- in building ----
            
        elif isinstance(table_id, list): # case multiple tables
            
            for i in range(len(table_id)):

                # catch title specific table 
                title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text

                # catch table
                table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[0][1:-1]

                # store table
                tables.append(table)
                # -----------

https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=CAP_LIB_P&aval=P&idGrupo=00000000020456&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=Grupo%20Tandem%20en%20Nano-bio-f%EDsica&codIdGrupo=COL0202529
CAP_LIB_P_TABLE
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=TES_MAST_P&aval=P&idGrupo=00000000020456&avalGr=T&codigoGrupo=&grCAv=2&nmeGrupo=Grupo%20Tandem%20en%20Nano-bio-f%EDsica&codIdGrupo=COL0202529
TM_P_TABLE
